In [2]:
%run ../imports.py

C:\Users\Jason\AppData\Local\conda\conda\envs\DSI-6\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


### Master Feature Types and Mapping from Raw to Readable Feature Names 

In [17]:
# All types of fouls with 100+ occurences over last 3 seasons, raw as written in play-by-play files
# Other rare types listed on: https://pudding.cool/2017/02/two-minute-report/
# (ex: C.P.Foul or Clear Path)
foul_types_raw_all = ['Personal','P.FOUL',
                      'Turnover: Shot Clock',
                      'S.FOUL','Shooting',
                      'L.B.FOUL',
                      'OFF.Foul',
                      'Traveling']

foul_types = ['Personal', '24 Second', 'Shooting', 'Loose Ball', 'Offensive', 'Traveling']

# Originally included more foul types, but processing a season's data took more than 4 hours
foul_types_dict = {'P.FOUL':'Personal', # 'Personal Take Foul':'Personal',
                   'Personal':'Personal', # 'Personal Block':'Personal',
                   'Turnover: Shot Clock':'24 Second',
                   'S.FOUL':'Shooting','Shooting':'Shooting',
                   'L.B.FOUL':'Loose Ball',
                   'OFF.Foul':'Offensive',
                   'Traveling':'Traveling'
                   #'T.Foul':'Technical','T.Foul (Def. 3 Sec':'Defensive 3 Second'
}

features = ['SCORE','HOMEDESCRIPTION','VISITORDESCRIPTION','EVENTNUM','GAME_ID','PCTIMESTRING',
            'PERIOD','PLAYER1_ID','PLAYER1_NAME','PLAYER1_TEAM_NICKNAME',
            'PLAYER2_ID','PLAYER2_NAME','PLAYER2_TEAM_NICKNAME',
            'PLAYER3_ID','PLAYER3_NAME','PLAYER3_TEAM_NICKNAME',
            'SCOREMARGIN'
           ]

team_to_abrev_dict = {'76ers':'PHI',
'Bucks':'MIL',
'Bulls':'CHI',
'Cavaliers':'CLE',
'Celtics':'BOS',
'Clippers':'LAC',
'Grizzlies':'MEM',
'Hawks':'ATL',
'Heat':'MIA',
'Hornets':'CHA',
'Jazz':'UTA',
'Kings':'SAC',
'Knicks':'NYK',
'Lakers':'LAL',
'Magic':'ORL',
'Mavericks':'DAL',
'Nets':'BKN',
'Nuggets':'DEN',
'Pacers':'IND',
'Pelicans':'NOP',
'Pistons':'DET',
'Raptors':'TOR',
'Rockets':'HOU',
'Spurs':'SAS',
'Suns':'PHX',
'Thunder':'OKC',
'Timberwolves':'MIN',
'Trail Blazers':'POR',
'Warriors':'GSW',
'Wizards':'WAS'}

### Checking Which Columns are Relevant in Seasonal Play-by-Play Data

In [4]:
# df = pd.read_csv('./data_raw/2015-16_pbp.csv')
# df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568333 entries, 0 to 568332
Data columns (total 34 columns):
Unnamed: 0                   568333 non-null int64
EVENTMSGACTIONTYPE           568333 non-null int64
EVENTMSGTYPE                 568333 non-null int64
EVENTNUM                     568333 non-null int64
GAME_ID                      568333 non-null int64
HOMEDESCRIPTION              294317 non-null object
NEUTRALDESCRIPTION           0 non-null float64
PCTIMESTRING                 568333 non-null object
PERIOD                       568333 non-null int64
PERSON1TYPE                  566172 non-null float64
PERSON2TYPE                  568333 non-null int64
PERSON3TYPE                  568333 non-null int64
PLAYER1_ID                   568333 non-null int64
PLAYER1_NAME                 517980 non-null object
PLAYER1_TEAM_ABBREVIATION    517070 non-null object
PLAYER1_TEAM_CITY            517070 non-null object
PLAYER1_TEAM_ID              517070 non-null float64
PLAYER1_TEAM_NIC

In [7]:
# df.head()

,Unnamed: 0,EVENTMSGACTIONTYPE,EVENTMSGTYPE,EVENTNUM,GAME_ID,HOMEDESCRIPTION,NEUTRALDESCRIPTION,PCTIMESTRING,PERIOD,PERSON1TYPE,PERSON2TYPE,PERSON3TYPE,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ABBREVIATION,PLAYER1_TEAM_CITY,PLAYER1_TEAM_ID,PLAYER1_TEAM_NICKNAME,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ABBREVIATION,PLAYER2_TEAM_CITY,PLAYER2_TEAM_ID,PLAYER2_TEAM_NICKNAME,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ABBREVIATION,PLAYER3_TEAM_CITY,PLAYER3_TEAM_ID,PLAYER3_TEAM_NICKNAME,SCORE,SCOREMARGIN,VISITORDESCRIPTION,WCTIMESTRING
0,0,0,12,0,21500758,NaN,NaN,12:00,1,0.0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9:11 PM
1,1,0,10,1,21500758,Jump Ball Gobert vs. Monroe: Tip to Middleton,NaN,12:00,1,4.0,5,5,203497,Rudy Gobert,UTA,Utah,1.610613e+09,Jazz,202328,Greg Monroe,MIL,Milwaukee,1.610613e+09,Bucks,203114,Khris Middleton,MIL,Milwaukee,1.610613e+09,Bucks,NaN,NaN,NaN,9:11 PM
2,2,79,2,2,21500758,NaN,NaN,11:45,1,5.0,0,0,203953,Jabari Parker,MIL,Milwaukee,1.610613e+09,Bucks,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MISS Parker 17' Pullup Jump Shot,9:11 PM
3,3,0,4,3,21500758,Neto REBOUND (Off:0 Def:1),NaN,11:44,1,4.0,0,0,203526,Raul Neto,UTA,Utah,1.610613e+09,Jazz,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9:11 PM
4,4,63,1,4,21500758,Neto 4' Fadeaway Jumper (2 PTS),NaN,11:19,1,4.0,0,0,203526,Raul Neto,UTA,Utah,1.610613e+09,Jazz,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0 - 2,2,NaN,9:11 PM


In [6]:
# df = df[features]
# df.to_csv('./data_raw/2015-16_pbp_gzip.csv', index=False, compression='gzip')

In [ ]:
df = pd.read_csv('./data_raw/2015-16_pbp_gzip.csv', compression='gzip')

In [15]:
df['PERSON1TYPE'].value_counts()

4.0    260063
5.0    257007
2.0     18343
3.0     18252
0.0     10104
1.0      2252
6.0        77
7.0        74
Name: PERSON1TYPE, dtype: int64

In [16]:
df['PERSON2TYPE'].value_counts() 
# These variables doesn't seem at all relevant, since we have names of each person 1-3

0    387193
5     90574
4     90565
7         1
Name: PERSON2TYPE, dtype: int64

In [17]:
df['PERSON3TYPE'].value_counts()

0    487350
1     66681
4      7404
5      6807
3        50
2        41
Name: PERSON3TYPE, dtype: int64

### Mapping Raw to Re-Named Features

In [13]:
# Only interested in play-by-play events that are fouls and also one of the 6 most common foul types
def process_pbp(df):
    c = 0
    for idx, row in df.iterrows():
        c += 1
        if c % 50000 == 0: print(c, ' rows processed')
        if type(row['VISITORDESCRIPTION']) == str: 
            for foul in foul_types_raw_all:
                if (foul in row['VISITORDESCRIPTION']):
                    df.loc[idx, 'Foul'] = foul
                    df.loc[idx, 'Fouler'] = row['PLAYER1_NAME']
                    df.loc[idx, 'Fouler_team'] = row['PLAYER1_TEAM_ABBREVIATION']
                    df.loc[idx, 'is_home_team'] = False
                    if type(row["PLAYER2_NAME"]) != float: # excluding null values
                        df.loc[idx, 'Foulee'] = row['PLAYER2_NAME']
                        df.loc[idx, 'Foulee_team'] = row['PLAYER2_TEAM_ABBREVIATION']

        if (type(row['HOMEDESCRIPTION']) == str):
            for foul in foul_types_raw_all:
                if (foul in row['HOMEDESCRIPTION']):
                    df.loc[idx, 'Foul'] = foul
                    df.loc[idx, 'Fouler'] = row['PLAYER1_NAME']
                    df.loc[idx, 'Fouler_team'] = row['PLAYER1_TEAM_ABBREVIATION']
                    df.loc[idx, 'is_home_team'] = True
                    if type(row["PLAYER2_NAME"]) != float: # excluding null values
                        df.loc[idx, 'Foulee'] = row['PLAYER2_NAME']
                        df.loc[idx, 'Foulee_team'] = row['PLAYER2_TEAM_ABBREVIATION']

In [30]:
process_pbp(df)
df.info()

50000  rows processed
100000  rows processed
150000  rows processed
200000  rows processed
250000  rows processed
300000  rows processed
350000  rows processed
400000  rows processed
450000  rows processed
500000  rows processed
550000  rows processed
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568333 entries, 0 to 568332
Data columns (total 24 columns):
SCORE                        146428 non-null object
HOMEDESCRIPTION              294317 non-null object
VISITORDESCRIPTION           290712 non-null object
EVENTNUM                     568333 non-null int64
GAME_ID                      568333 non-null int64
PCTIMESTRING                 568333 non-null object
PERIOD                       568333 non-null int64
PLAYER1_ID                   568333 non-null int64
PLAYER1_NAME                 517980 non-null object
PLAYER1_TEAM_ABBREVIATION    517070 non-null object
PLAYER2_ID                   568333 non-null int64
PLAYER2_NAME                 181139 non-null object
PLAYER2_TEAM_ABBRE

In [31]:
df.drop('Home_team', axis=1, inplace=True)

In [32]:
df.to_csv('./data/play_by_play.csv', index=False, compression='gzip')

### Engineering 2016-2017 Season Dataset

In [7]:
df2 = pd.read_csv('./data_raw/2016-17_pbp.csv')
df2.info()

C:\Users\Jason\AppData\Local\conda\conda\envs\DSI-6\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (71,89,90) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567365 entries, 0 to 567364
Columns: 111 entries, Unnamed: 0 to AWAY_PLAYER_ID_5_PLAY_TIME
dtypes: float64(53), int64(24), object(34)
memory usage: 480.5+ MB


In [9]:
# df2.head()

,Unnamed: 0,EVENTMSGACTIONTYPE,EVENTMSGTYPE,EVENTNUM,GAME_ID,HOMEDESCRIPTION,NEUTRALDESCRIPTION,PCTIMESTRING,PERIOD,PERSON1TYPE,PERSON2TYPE,PERSON3TYPE,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ABBREVIATION,PLAYER1_TEAM_CITY,PLAYER1_TEAM_ID,PLAYER1_TEAM_NICKNAME,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ABBREVIATION,PLAYER2_TEAM_CITY,PLAYER2_TEAM_ID,PLAYER2_TEAM_NICKNAME,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ABBREVIATION,PLAYER3_TEAM_CITY,PLAYER3_TEAM_ID,PLAYER3_TEAM_NICKNAME,SCORE,SCOREMARGIN,VISITORDESCRIPTION,WCTIMESTRING,HOME_TEAM,AWAY_TEAM,HOME_SCORE,AWAY_SCORE,TIME,TEAM,TYPE,SUB_TYPE,ASSIST_PLAYER_ID,ASSIST_COUNT,BLOCK_PLAYER_ID,...,DOUBLE_TECHNICAL_PLAYER_ID_2,DOUBLE_PERSONAL_PLAYER_ID_1,DOUBLE_PERSONAL_PLAYER_ID_2,DOUBLE_PERSONAL_FOUL_COUNT_1,DOUBLE_PERSONAL_FOUL_COUNT_2,COACH_TECHNICAL,SHOT_PLAYER_ID,POINTS_SCORED,TOTAL_POINTS_SCORED,SHOT_MADE,SHOT_TYPE,STEAL_PLAYER_ID,STEAL_COUNT,TURNOVER_PLAYER_ID,TURNOVER_COUNT,TEAM_TURNOVER_COUNT,TURNOVER_TYPE,POSSESSION_ID,PERIOD_START,PERIOD_END,TIMEOUT_TYPE,VIOLATION,VIOLATION_PLAYER_ID,EJECTION,EJECTION_PLAYER_ID,HOME_PLAYER_ID_1,HOME_PLAYER_ID_2,HOME_PLAYER_ID_3,HOME_PLAYER_ID_4,HOME_PLAYER_ID_5,AWAY_PLAYER_ID_1,AWAY_PLAYER_ID_2,AWAY_PLAYER_ID_3,AWAY_PLAYER_ID_4,AWAY_PLAYER_ID_5,HOME_PLAYER_ID_1_PLAY_TIME,HOME_PLAYER_ID_2_PLAY_TIME,HOME_PLAYER_ID_3_PLAY_TIME,HOME_PLAYER_ID_4_PLAY_TIME,HOME_PLAYER_ID_5_PLAY_TIME,AWAY_PLAYER_ID_1_PLAY_TIME,AWAY_PLAYER_ID_2_PLAY_TIME,AWAY_PLAYER_ID_3_PLAY_TIME,AWAY_PLAYER_ID_4_PLAY_TIME,AWAY_PLAYER_ID_5_PLAY_TIME
0,0,0,12,0,21600256,NaN,NaN,12:00,1,0.0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7:42 PM,Celtics,Heat,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,201609,203585,202355,202682,202337,202738,202340,203092,101161,203109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,10,1,21600256,Jump Ball Whiteside vs. Johnson: Tip to Zeller,NaN,12:00,1,4.0,5,5,202355,Hassan Whiteside,MIA,Miami,1.610613e+09,Heat,101161,Amir Johnson,BOS,Boston,1.610613e+09,Celtics,203092,Tyler Zeller,BOS,Boston,1.610613e+09,Celtics,NaN,NaN,NaN,7:42 PM,Celtics,Heat,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201609,203585,202355,202682,202337,202738,202340,203092,101161,203109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,71,2,2,21600256,NaN,NaN,11:38,1,5.0,0,0,202340,Avery Bradley,BOS,Boston,1.610613e+09,Celtics,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MISS Bradley 2' Finger Roll Layup,7:42 PM,Celtics,Heat,0,0,22,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,202340.0,NaN,NaN,False,Layup,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201609,203585,202355,202682,202337,202738,202340,203092,101161,203109,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0
3,3,0,4,3,21600256,Williams REBOUND (Off:0 Def:1),NaN,11:37,1,4.0,0,0,202682,Derrick Williams,MIA,Miami,1.610613e+09,Heat,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7:42 PM,Celtics,Heat,0,0,23,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201609,203585,202355,202682,202337,202738,202340,203092,101161,203109,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0,23.0
4,4,47,2,4,21600256,MISS Williams 10' Turnaround Jump Shot,NaN,11:22,1,4.0,0,0,202682,Derrick Williams,MIA,Miami,1.610613e+09,Heat,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7:43 PM,Celtics,Heat,0,0,38,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,202682.0,NaN,NaN,False,Shot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201609,203585,202355,202682,202337,202738,202340,203092,101161,203109,38.0,38.0,38.0,38.0,38.0,38.0,38.0,38.0,38.0,38.0


In [10]:
# df2 = df2[features]
# df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567365 entries, 0 to 567364
Data columns (total 17 columns):
SCORE                        148830 non-null object
HOMEDESCRIPTION              293104 non-null object
VISITORDESCRIPTION           289705 non-null object
EVENTNUM                     567365 non-null int64
GAME_ID                      567365 non-null int64
PCTIMESTRING                 567365 non-null object
PERIOD                       567365 non-null int64
PLAYER1_ID                   567365 non-null int64
PLAYER1_NAME                 516896 non-null object
PLAYER1_TEAM_ABBREVIATION    515854 non-null object
PLAYER2_ID                   567365 non-null int64
PLAYER2_NAME                 180677 non-null object
PLAYER2_TEAM_ABBREVIATION    180677 non-null object
PLAYER3_ID                   567365 non-null int64
PLAYER3_NAME                 13730 non-null object
PLAYER3_TEAM_ABBREVIATION    13730 non-null object
SCOREMARGIN                  148830 non-null object
dtypes: int64

In [14]:
process_pbp(df2)

50000  rows processed
100000  rows processed
150000  rows processed
200000  rows processed
250000  rows processed
300000  rows processed
350000  rows processed
400000  rows processed
450000  rows processed
500000  rows processed
550000  rows processed


In [11]:
# df2.to_csv('./data_raw/2016-17_pbp_gzip.csv', index=False, compression='gzip')

In [46]:
df2 = pd.read_csv('./data_raw/2016-17_pbp_gzip.csv', compression='gzip')

In [19]:
# df2.drop(['PLAYER1_TEAM_ABBREVIATION','PLAYER2_TEAM_ABBREVIATION','PLAYER3_TEAM_ABBREVIATION'], 
#          axis=1, inplace=True)

# df3 = pd.read_csv('./data_raw/2016-17_pbp_gzip.csv', compression='gzip')

In [47]:
# df2 = pd.concat([df2, df3[['PLAYER1_TEAM_NICKNAME','PLAYER2_TEAM_NICKNAME','PLAYER3_TEAM_NICKNAME']]], 
#                  axis=1)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567365 entries, 0 to 567364
Data columns (total 17 columns):
SCORE                    148830 non-null object
HOMEDESCRIPTION          293104 non-null object
VISITORDESCRIPTION       289705 non-null object
EVENTNUM                 567365 non-null int64
GAME_ID                  567365 non-null int64
PCTIMESTRING             567365 non-null object
PERIOD                   567365 non-null int64
PLAYER1_ID               567365 non-null int64
PLAYER1_NAME             516896 non-null object
PLAYER1_TEAM_NICKNAME    515854 non-null object
PLAYER2_ID               567365 non-null int64
PLAYER2_NAME             180677 non-null object
PLAYER2_TEAM_NICKNAME    180677 non-null object
PLAYER3_ID               567365 non-null int64
PLAYER3_NAME             13730 non-null object
PLAYER3_TEAM_NICKNAME    13730 non-null object
SCOREMARGIN              148830 non-null object
dtypes: int64(6), object(11)
memory usage: 73.6+ MB


In [53]:
df3['GAME_ID'][df3['GAME_ID']<21509999].value_counts()

21500391    707
21500022    640
21500336    616
21500674    597
21500819    593
21501131    582
21500792    570
21500615    570
21500297    570
21500261    567
21500558    564
21500624    561
21500814    561
21500141    556
21500967    555
21500834    554
21500948    552
21500311    549
21500270    547
21500085    544
21500843    543
21500024    543
21500401    543
21500933    542
21500483    541
21500170    539
21500777    539
21500599    538
21500166    536
21500515    534
21500616    534
21500701    534
21501037    533
21500034    533
21500523    533
21500951    532
21500038    532
21500025    530
21500728    529
21500623    529
21500274    528
21500319    528
21500992    527
21500977    527
21501012    527
21500627    525
21500161    525
21500635    525
21500587    525
21500560    524
21500604    523
21500755    523
21500679    522
21500478    522
21500467    522
21500198    521
21500359    521
21500202    521
21500832    521
21500014    520
21500874    520
21501157    520
21501121

In [54]:
df3['GAME_ID'][df3['GAME_ID']>21599999].value_counts()

21600711    691
21600281    621
21600261    614
21600212    592
21600422    583
21601045    581
21600830    573
21600021    572
21600033    572
21600478    565
21600372    563
21600005    556
21601177    554
21600784    552
21601067    550
21601197    549
21600948    549
21601150    548
21601076    547
21600185    547
21600359    547
21600727    545
21601088    545
21600650    544
21600063    543
21600195    543
21600017    543
21600500    543
21600378    543
21600310    541
21600554    540
21600270    540
21601112    540
21600075    538
21600125    537
21600333    537
21600929    537
21600235    536
21600963    533
21600269    533
21600181    533
21600292    531
21600424    531
21600144    531
21600610    530
21600648    529
21600803    527
21600529    527
21601106    526
21601205    526
21600550    525
21600092    525
21600508    525
21600856    524
21601153    523
21601003    523
21600070    522
21600157    521
21600680    521
21600872    521
21600881    520
21600364    520
21600519

In [55]:
df2['GAME_ID'][df2['GAME_ID']<21509999].value_counts()

Series([], Name: GAME_ID, dtype: int64)

In [56]:
df2['GAME_ID'][df2['GAME_ID']>21599999].value_counts()

21600711    691
21600281    621
21600261    614
21600212    592
21600422    583
21601045    581
21600830    573
21600021    572
21600033    572
21600478    565
21600372    563
21600005    556
21601177    554
21600784    552
21601067    550
21601197    549
21600948    549
21601150    548
21601076    547
21600185    547
21600359    547
21600727    545
21601088    545
21600650    544
21600063    543
21600195    543
21600017    543
21600500    543
21600378    543
21600310    541
21600554    540
21600270    540
21601112    540
21600075    538
21600125    537
21600333    537
21600929    537
21600235    536
21600963    533
21600269    533
21600181    533
21600292    531
21600424    531
21600144    531
21600610    530
21600648    529
21600803    527
21600529    527
21601106    526
21601205    526
21600550    525
21600092    525
21600508    525
21600856    524
21601153    523
21601003    523
21600070    522
21600157    521
21600680    521
21600872    521
21600881    520
21600364    520
21600519

In [59]:
# df3.to_csv('./data_raw/2015-16_pbp_gzip.csv', index=False, compression='gzip')

In [ ]:
df3 = df3.append(df2, ignore_index=True)

In [26]:
# Both seasons data appended together (2016-17 and 2015-16)
df3.to_csv('./data/play_by_play.csv', index=False, compression='gzip')

In [65]:
df = None
df2 = None
df3 = pd.read_csv('./data/play_by_play.csv', compression='gzip')

In [66]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1135698 entries, 0 to 1135697
Data columns (total 23 columns):
SCORE                    295258 non-null object
HOMEDESCRIPTION          587421 non-null object
VISITORDESCRIPTION       580417 non-null object
EVENTNUM                 1135698 non-null int64
GAME_ID                  1135698 non-null int64
PCTIMESTRING             1135698 non-null object
PERIOD                   1135698 non-null int64
PLAYER1_ID               1135698 non-null int64
PLAYER1_NAME             1034876 non-null object
PLAYER2_ID               1135698 non-null int64
PLAYER2_NAME             361816 non-null object
PLAYER3_ID               1135698 non-null int64
PLAYER3_NAME             27941 non-null object
SCOREMARGIN              295258 non-null object
Foul                     102457 non-null object
Fouler                   99820 non-null object
Fouler_team              99820 non-null object
is_home_team             102342 non-null object
Foulee                  

In [76]:
df3['Fouler_team'].value_counts().sort_index()

ATL    3069
BKN    3232
BOS    3512
CHA    2894
CHI    3024
CLE    3133
DAL    3196
DEN    3344
DET    3058
GSW    3334
HOU    3429
IND    3227
LAC    3387
LAL    3390
MEM    3653
MIA    3241
MIL    3440
MIN    3388
NOP    3251
NYK    3391
OKC    3432
ORL    3344
PHI    3621
PHX    3887
POR    3532
SAC    3368
SAS    3015
TOR    3316
UTA    3250
WAS    3462
Name: Fouler_team, dtype: int64

In [75]:
df3['PLAYER1_TEAM_NICKNAME'].value_counts().sort_index()

76ers            35576
Bucks            32941
Bulls            34446
Cavaliers        33778
Celtics          35246
Clippers         34660
Grizzlies        34612
Hawks            35196
Heat             33779
Hornets          33298
Jazz             34194
Kings            34236
Knicks           34193
Lakers           34155
Magic            34009
Mavericks        33561
Nets             34611
Nuggets          34897
Pacers           34083
Pelicans         33968
Pistons          34079
Raptors          33810
Rockets          35642
Spurs            34047
Suns             35745
Thunder          36231
Timberwolves     33726
Trail Blazers    35010
Warriors         35077
Wizards          34118
Name: PLAYER1_TEAM_NICKNAME, dtype: int64

### Engineering Adjusted Player Usage Variable, Counted Per Game

In [ ]:
# Almost all Types of Shots and their Frequency:
# Jump Shot                             0.461887
# Layup Shot                            0.103953
# Driving Layup Shot                    0.066412
# Pullup Jump shot                      0.049389
# Floating Jump shot                    0.024552
# Hook Shot                             0.023900
# Tip Layup Shot                        0.022701
# Step Back Jump shot                   0.019193
# Running Layup Shot                    0.018051
# Dunk Shot                             0.017982
# Turnaround Jump Shot                  0.016326
# Cutting Layup Shot                    0.015823
# Fadeaway Jump Shot                    0.012487
# Driving Finger Roll Layup Shot        0.011093
# Reverse Layup Shot                    0.010899
# Putback Layup Shot                    0.010773
# Running Jump Shot                     0.008614
# Jump Bank Shot                        0.008580
# Turnaround Hook Shot                  0.008112
# Driving Floating Jump Shot            0.008077
# Alley Oop Dunk Shot                   0.007655
# Driving Dunk Shot                     0.007380
# Driving Reverse Layup Shot            0.007369
# Turnaround Fadeaway shot              0.006889
# Cutting Dunk Shot                     0.006649
# Running Dunk Shot                     0.006238
# Driving Hook Shot                     0.004776
# Alley Oop Layup shot                  0.004684
# Driving Bank shot                     0.004536
# Putback Dunk Shot                     0.003622
# Finger Roll Layup Shot                0.003496
# Running Finger Roll Layup Shot        0.002491
# Driving Floating Bank Jump Shot       0.001954
# Cutting Finger Roll Layup Shot        0.001748
# Turnaround Bank shot                  0.001599
# Running Reverse Layup Shot            0.001554
# Tip Dunk Shot                         0.001291
# Pullup Bank shot                      0.001280
# Running Pull-Up Jump Shot             0.001234

In [34]:
match = re.search("(\d+)\'", "Neto 4' Fadeaway Jumper (2 PTS)")
match.group()[0]

'4'

In [29]:
df3.head()

,SCORE,HOMEDESCRIPTION,VISITORDESCRIPTION,EVENTNUM,GAME_ID,PCTIMESTRING,PERIOD,PLAYER1_ID,PLAYER1_NAME,PLAYER2_ID,PLAYER2_NAME,PLAYER3_ID,PLAYER3_NAME,SCOREMARGIN,Foul,Fouler,Fouler_team,is_home_team,Foulee,Foulee_team,PLAYER1_TEAM_NICKNAME,PLAYER2_TEAM_NICKNAME,PLAYER3_TEAM_NICKNAME
0,NaN,NaN,NaN,0,21500758,12:00,1,0,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Jump Ball Gobert vs. Monroe: Tip to Middleton,NaN,1,21500758,12:00,1,203497,Rudy Gobert,202328,Greg Monroe,203114,Khris Middleton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jazz,Bucks,Bucks
2,NaN,NaN,MISS Parker 17' Pullup Jump Shot,2,21500758,11:45,1,203953,Jabari Parker,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bucks,NaN,NaN
3,NaN,Neto REBOUND (Off:0 Def:1),NaN,3,21500758,11:44,1,203526,Raul Neto,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jazz,NaN,NaN
4,0 - 2,Neto 4' Fadeaway Jumper (2 PTS),NaN,4,21500758,11:19,1,203526,Raul Neto,0,NaN,0,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,Jazz,NaN,NaN


In [42]:
df3.loc[4,'SCORE']

4    0 - 2
4      NaN
Name: SCORE, dtype: object

In [81]:
usage_types1 = ['Layup', 'Dunk', 'Driving'] # Common situations to foul, x2 weight
usage_types2 = ['Shot', 'shot', 'Jumper', 'Turnover', 'REBOUND']
usage_type3 = 'AST' # only type which co-occurs during another event type
# shot_types = ['Layup', 'Dunk', 'Driving', 'Shot', 'shot', 'Jumper'] # All have distance shot from
# Specific type not needed since it takes all remaining shot types including: 
# Jump shots, pullups, floaters, hooks, step backs, bank shot
# NBA Usage statistic formula:
# 100 * ((FGA + 0.44 * FTA + TOV) * (Team MP / 5)) / (MP * (Team FGA + 0.44 * Team FTA + Team TOV))

# Above are all the considered types of opportunities where there could be a foul called
# for the player with the ball (included later are all on-ball foul types)
# (each type has a different baseline foul rate but here we assume an overall equal average)
# Types of Turnovers included: Travel, Lost Ball, Bad Pass, Out of Bounds, Foul Turnover, 3 Second

In [38]:
test = pd.DataFrame(data=[[1,2],[3,4]])
test

,0,1
0,1,2
1,3,4


In [41]:
plus1(test, 2, 2)
test

,0,1,2
0,1.0,2.0,NaN
1,3.0,4.0,NaN
2,NaN,NaN,1.0


In [73]:
def plus1(df, row_index, col_name):
    try:
        df.loc[row_index, col_name] += 1
    except KeyError:
        df.loc[row_index, col_name] = 1

def make_usage(df, df_usage):
    
    c = 0
    for idx, row in df.iterrows():
        c += 1
        if c % 50000 == 0: print(c, ' rows processed')

        if type(row['Foul']) == float:
        
            usage1done = False
            usage2done = False
            if type(row['VISITORDESCRIPTION']) == str: 
                text = row['VISITORDESCRIPTION']
                is_home_text = 'visitor'
                player2text = 'home'
            elif (type(row['HOMEDESCRIPTION']) == str):
                text = row['HOMEDESCRIPTION']
                is_home_text = 'home'
                player2text = 'home'
            else:
                text = ''
                usage1done = True
                usage2done = True                
                is_home_text = 'error'
                player2text = 'error'
            
            for usage in usage_types1:
                if (usage in text) & (not usage1done):
                    usage1done = True
                    plus1(df_usage, row['GAME_ID'], str(row['PLAYER1_NAME']) + '_AdjUsage')
                    # Plus 1 twice due to double weight for driving shots
                    plus1(df_usage, row['GAME_ID'], str(row['PLAYER1_NAME']) + '_AdjUsage')
                    match = re.search("(\d+)\'", text)
                    try:
                        shot_dist = int(match.group()[0])
                        if shot_dist <= 5:
                            plus1(df_usage, row['GAME_ID'], str(row['PLAYER1_NAME']) + '_<5ft')
                        elif shot_dist <= 10:
                            plus1(df_usage, row['GAME_ID'], str(row['PLAYER1_NAME']) + '_5-10ft')
                        elif shot_dist <= 15:
                            plus1(df_usage, row['GAME_ID'], str(row['PLAYER1_NAME']) + '_10-15ft')
                        elif shot_dist <= 22:
                            plus1(df_usage, row['GAME_ID'], str(row['PLAYER1_NAME']) + '_15-22ft')
                        else:
                            plus1(df_usage, row['GAME_ID'], str(row['PLAYER1_NAME']) + '_23+ft')
                    except: pass

            if 'AST' in text:
                plus1(df_usage, row['GAME_ID'], str(row['PLAYER2_NAME']) + '_AdjUsage')
                
            if not usage1done:
                for usage in usage_types2:
                    if (usage in text) & (not usage2done):
                        usage2done = True
                        plus1(df_usage, row['GAME_ID'], str(row['PLAYER1_NAME']) + '_AdjUsage')

        elif type(row['Foul']) == str:
            foul = row['Foul']
            if foul in ['Personal', 'Shooting', 'Loose Ball']:
                plus1(df_usage, row['GAME_ID'], str(row['Foulee']))
                plus1(df_usage, row['GAME_ID'], str(row['Foulee_team']) + '_' + foul+player2text)
                plus1(df_usage, row['GAME_ID'], str(row['Fouler_team']) + '_'+foul+'_given_'+is_home_text)
                plus1(df_usage, row['GAME_ID'], str(row['Foulee']) + '_AdjUsage')
            elif foul in ['Offensive', 'Traveling']:
                plus1(df_usage, row['GAME_ID'], str(row['Fouler']))
                plus1(df_usage, row['GAME_ID'], str(row['Fouler_team']) + '_' + foul+is_home_text)
                plus1(df_usage, row['GAME_ID'], str(row['Foulee_team']) + '_'+foul+'_taken'+player2text)
                plus1(df_usage, row['GAME_ID'], str(row['Fouler']) + '_AdjUsage')
            elif foul == '24 Second':
                plus1(df_usage, row['GAME_ID'], str(row['Fouler_team']) + '_24_Sec'+is_home_text)
                

In [1]:
df_usage = pd.DataFrame()

make_usage(df3, df_usage)

NameError: name 'pd' is not defined

In [105]:
df_usage.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2460 entries, 21500758 to 21600130
Columns: 2464 entries, Jabari Parker_AdjUsage to Joe Harris_5-10ft
dtypes: float64(2464)
memory usage: 46.3 MB


In [107]:
df_usage.head()

,Jabari Parker_AdjUsage,Raul Neto_AdjUsage,Khris Middleton_AdjUsage,Rodney Hood_AdjUsage,Rudy Gobert_AdjUsage,Michael Carter-Williams_AdjUsage,Michael Carter-Williams_<5ft,Greg Monroe_AdjUsage,Derrick Favors_AdjUsage,Raul Neto_<5ft,Khris Middleton_<5ft,Gordon Hayward_AdjUsage,Giannis Antetokounmpo_AdjUsage,Giannis Antetokounmpo_<5ft,Derrick Favors_<5ft,Jabari Parker_<5ft,Chris Johnson_AdjUsage,Jerryd Bayless_AdjUsage,Johnny O'Bryant III_AdjUsage,Miles Plumlee_AdjUsage,Trey Burke_AdjUsage,Rodney Hood_<5ft,Tyler Ennis_AdjUsage,Trevor Booker_AdjUsage,Trevor Booker_<5ft,Johnny O'Bryant III_<5ft,Trey Lyles_AdjUsage,Rashad Vaughn_AdjUsage,Rashad Vaughn_<5ft,Jerryd Bayless,MIL_Travelingvisitor,nan_Traveling_takenhome,Greg Monroe_<5ft,Joe Ingles_AdjUsage,Chris Johnson_<5ft,Khris Middleton,Michael Carter-Williams_5-10ft,Gordon Hayward_<5ft,Justin Holiday_AdjUsage,James Johnson_AdjUsage,Kyle Lowry_AdjUsage,Kyle Lowry_<5ft,Nikola Mirotic_AdjUsage,Jonas Valanciunas_AdjUsage,Jimmy Butler_AdjUsage,...,Patricio Garino_AdjUsage,Marcus Georges-Hunt_AdjUsage,Gary Payton II_<5ft,Maurice Ndour,Larry Sanders_<5ft,David Nwaba_5-10ft,Marcus Georges-Hunt_<5ft,Marcus Georges-Hunt,Gary Payton II,DeAndre' Bembry_5-10ft,Maurice Ndour_5-10ft,Wayne Selden_5-10ft,Chinanu Onuaku,Bryn Forbes_5-10ft,Georgios Papagiannis,Marcus Georges-Hunt_5-10ft,Isaiah Taylor,Salah Mejri_5-10ft,Edy Tavares_5-10ft,Dahntay Jones,Sheldon McClellan_5-10ft,Nicolas Brussino,Jarrod Uthoff_5-10ft,Jarrod Uthoff,Quinn Cook,Tim Quarterman,SAC_Personalerror,PHX_Personal_given_error,Mike Dunleavy_5-10ft,SAS_Travelingerror,Justin Hamilton_5-10ft,John Lucas III_AdjUsage,John Lucas III_<5ft,Andrew Nicholson_5-10ft,Demetrius Jackson_<5ft,DEN_Travelingerror,Nicolas Laprovittola_5-10ft,AJ Hammons_<5ft,Danuel House_AdjUsage,Rodney McGruder,Pat Connaughton_5-10ft,DeAndre Liggins_5-10ft,Thabo Sefolosha_5-10ft,Ryan Kelly_5-10ft,Joe Harris_5-10ft
21500758,14.0,16.0,35.0,23.0,20.0,35.0,7.0,22.0,34.0,2.0,2.0,24.0,21.0,4.0,4.0,2.0,7.0,14.0,10.0,9.0,4.0,2.0,2.0,7.0,2.0,1.0,9.0,2.0,1.0,1.0,2.0,2.0,2.0,6.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21500991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,5.0,52.0,11.0,25.0,7.0,36.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21500538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21500947,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21500095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
df_usage.to_csv('./data/usage.csv', index=True, compression='gzip')